In [2]:
import pandas as pd

print("reading data")
df = pd.read_csv('../scripts/dados_ouvidoria.csv',sep=';',error_bad_lines=False,encoding='utf-8')

reading data


b'Skipping line 351863: expected 9 fields, saw 10\nSkipping line 352218: expected 9 fields, saw 10\nSkipping line 352407: expected 9 fields, saw 10\nSkipping line 352781: expected 9 fields, saw 10\nSkipping line 355320: expected 9 fields, saw 10\nSkipping line 355611: expected 9 fields, saw 10\nSkipping line 355866: expected 9 fields, saw 10\nSkipping line 356001: expected 9 fields, saw 10\nSkipping line 356105: expected 9 fields, saw 10\nSkipping line 356724: expected 9 fields, saw 10\nSkipping line 357092: expected 9 fields, saw 10\nSkipping line 357664: expected 9 fields, saw 10\nSkipping line 357717: expected 9 fields, saw 10\nSkipping line 357792: expected 9 fields, saw 10\nSkipping line 357859: expected 9 fields, saw 10\nSkipping line 357916: expected 9 fields, saw 10\nSkipping line 357953: expected 9 fields, saw 10\nSkipping line 358113: expected 9 fields, saw 10\nSkipping line 358376: expected 9 fields, saw 10\nSkipping line 358397: expected 9 fields, saw 10\nSkipping line 3584

In [3]:
print("cleaning")
df['DATA_PERGUNTA'] = pd.to_datetime(df['DATA_PERGUNTA'], errors='coerce')

df.dropna()

df['MM'] = (df['DATA_PERGUNTA'].dt.month)

df['DD'] = (df['DATA_PERGUNTA'].dt.day)

df_18 = df[df.DATA_PERGUNTA.dt.year == 2018]

df_17 = df[df.DATA_PERGUNTA.dt.year == 2017]

# df_16 = df[df.DATA_PERGUNTA.dt.year == 2016]

frames = [df_17, df_18]

df_train = pd.concat(frames)


df_test = df[df.DATA_PERGUNTA.dt.year == 2019 ]

cleaning


In [4]:
print("axis")
# X_tr= df_train.DATA_PERGUNTA
X_tr= df_train.MM
# X_tr= df_train.drop(axis=0, columns=['MM', 'DD'])#.astype(float)#df_train.MM_DD
Y_tr = df_train.TIPO

# X_te = df_test.DATA_PERGUNTA
X_te = df_test.MM
# X_te = df_test.drop(axis=0, columns=['MM', 'DD'])#.astype(float)#df_train.MM_DD
Y_te = df_test.TIPO

# X = df.DATA_PERGUNTA
# Y = df.ASSUNTO

axis


In [5]:
print("test/train")
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, Y,  test_size=0.33, random_state=42)

# X_train= X_train.values.reshape(-1, 1)
# y_train= y_train.values.reshape(-1, 1)
# X_test = X_test.values.reshape(-1, 1)
# y_test = y_test.values.reshape(-1, 1)

X_train= X_tr.values.reshape(-1, 1)
# X_train= X_tr
y_train= Y_tr.values.reshape(-1, 1)
X_test = X_te.values.reshape(-1, 1)
# X_test = X_te
y_test = Y_te.values.reshape(-1, 1)

test/train


In [7]:
print("training model")
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np

extra_tree = ExtraTreeClassifier(random_state=0)
bag = BaggingClassifier(extra_tree, random_state=0).fit( X_train, np.ravel(y_train,order='C'))

dec_tr = DecisionTreeClassifier(random_state=0)
dec_tr.fit(X_train, np.ravel(y_train,order='C'))

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, np.ravel(y_train,order='C'))

rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.fit(X_train, np.ravel(y_train,order='C'))

print("trained")

training model
trained


In [8]:
classes = bag.classes_

def unique(list1):
 
    # intilize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(np.array([x.tolist()]))
    print(unique_list)
    return unique_list
    # print list
#     for x in unique_list:
#         print x,

def return_proba(classifier, class_index, mes):
    proba = classifier.predict_proba(mes)
    arr = proba.tolist()
    index=(int(class_index[0]))
    prb =((arr[0][index]))
    return prb

bag_array = []
for mes in unique(X_test):
#     print(mes)
    for class_name in classes:
        class_index = np.where(bag.classes_ == class_name)
        
        obj = {
            "Tipo": class_name,
            "Bagging": return_proba(bag, class_index, mes),
            "KNN": return_proba(knn, class_index, mes),
            "Decision Tree": return_proba(dec_tr, class_index, mes),
            "Random Forest": return_proba(rf, class_index, mes),
            "Mes": int(mes.tolist()[0][0])
        }
        print(obj)
#     print((proba[0]))
        bag_array.append(obj)

[array([[1.]]), array([[2.]]), array([[3.]]), array([[4.]]), array([[5.]]), array([[6.]]), array([[7.]]), array([[8.]]), array([[9.]]), array([[10.]]), array([[11.]]), array([[12.]])]
{'Tipo': 'Denúncia', 'Bagging': 0.16448673927644714, 'KNN': 0.0, 'Decision Tree': 0.16491715103669646, 'Random Forest': 0.1648781773950382, 'Mes': 1}
{'Tipo': 'Elogio', 'Bagging': 0.014959837234691884, 'KNN': 0.0, 'Decision Tree': 0.015094994361065326, 'Random Forest': 0.013166999559418642, 'Mes': 1}
{'Tipo': 'Informação', 'Bagging': 0.002371004116487232, 'KNN': 0.3333333333333333, 'Decision Tree': 0.002342326711199792, 'Random Forest': 0.0009018217564372297, 'Mes': 1}
{'Tipo': 'NÃO CLASSIFICADO', 'Bagging': 0.0023810931490844812, 'KNN': 0.0, 'Decision Tree': 0.0025158323935108875, 'Random Forest': 0.005032412047504605, 'Mes': 1}
{'Tipo': 'Reclamação', 'Bagging': 0.269145811230418, 'KNN': 0.6666666666666666, 'Decision Tree': 0.27153639281686476, 'Random Forest': 0.27304312271639597, 'Mes': 1}
{'Tipo': 'So

In [64]:
proba = bag.predict_proba(X_test)[:,0]
arr_true_0_indices = (y_test == 0.0)
arr_true_1_indices = (y_test == 1.0)

arr_pred_0 = proba[arr_true_0_indices]
arr_pred_1 = proba[arr_true_1_indices]

plt.hist(arr_pred_0, bins=40, label='True class 0', normed=True, histtype='step')
plt.hist(arr_pred_1, bins=40, label='True class 1', normed=True, histtype='step')
plt.xlabel('Network output')
plt.ylabel('Arbitrary units / probability')
plt.legend(loc='best')
plt.show()

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [73]:
for probs, pred in zip(bag.predict_proba(X_test), bag.predict(X_test)):
    print(probs[pred])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [83]:
classes = bag.classes_

bag_array = []

for class_name in classes:
    class_index = np.where(bag.classes_ == class_name)
    proba = bag.predict_proba(X_test)[class_index]
    arr = proba.tolist()
    index=(int(class_index[0]))
    prb =((arr[0][index]))
    obj = {
        "class": class_name,
        "proba": prb
    }
    print(obj)
#     print((proba[0]))
#     bag_array.append(obj)

{'class': 'Denúncia', 'proba': 0.16448673927644714}
{'class': 'Elogio', 'proba': 0.014959837234691884}
{'class': 'Informação', 'proba': 0.002371004116487232}
{'class': 'NÃO CLASSIFICADO', 'proba': 0.0023810931490844812}
{'class': 'Reclamação', 'proba': 0.269145811230418}
{'class': 'Solicitação', 'proba': 0.5350867506346431}
{'class': 'Sugestão', 'proba': 0.011568764358228286}


In [84]:
(X_test)

array([[1.],
       [1.],
       [1.],
       ...,
       [6.],
       [6.],
       [6.]])

In [1]:
df_prob = pd.DataFrame.from_dict(bag_array)
df_prob

NameError: name 'pd' is not defined

In [157]:
import plotly.express as px
import plotly.graph_objects as go


# fig = px.scatter(df_prob, x='Tipo', y='Probabilidade', color="Mes")
fig = px.scatter_matrix(df_prob, dimensions=["Bagging", "KNN", "Decision Tree", "Random Forest"], color="Tipo")
fig.write_html('TIPO_Compare_all_models.html', auto_open=True)

In [158]:
import plotly.express as px
import plotly.graph_objects as go

models_names = ["Bagging", "KNN", "Decision Tree", "Random Forest"]

for mn in models_names:
    fig = px.line(df_prob, x="Mes", y=f"{mn}", color="Tipo", line_group="Tipo", hover_name="Tipo",
            line_shape="spline", render_mode="svg")
    fig.write_html(f"TIPO_Proba_{mn}.html", auto_open=True)

In [160]:
import plotly.express as px
import plotly.graph_objects as go

models_names = ["Bagging", "KNN", "Decision Tree", "Random Forest"]

for mn in models_names:
    fig = px.line(df_prob, x="Tipo", y=f"{mn}", color="Mes", line_group="Mes", hover_name="Mes",
            line_shape="spline", render_mode="svg")
    fig.write_html(f"TIPO_Proba_{mn}_month_vision.html", auto_open=True)

In [151]:
import plotly.express as px

df_prob["e"] = df_prob["Probabilidade Bagging"]/100
fig = px.scatter(df_prob, x="Mes", y="Probabilidade Bagging", color="Tipo",
                 error_x="e", error_y="e")
fig.write_html('erros.html', auto_open=True)

In [9]:
print("bagging")
print(bag.score(X_test, y_test))

print('dec_tr')
print(dec_tr.score(X_test, y_test))

print('knn')
print(knn.score(X_test, y_test))

print('rf')
print(rf.score(X_test, y_test))


bagging
0.5569371325562816
dec_tr
0.5569371325562816
knn
0.3837187864883135
rf
0.5569371325562816
